[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/healthcare/02_Drug_Interactions_Analysis.ipynb)

# Drug Interactions Analysis - Ontology & Reasoning

## Overview

This notebook demonstrates **drug interactions analysis** using Semantica with focus on **multi-source correlation**, **safety ontology**, and **interaction detection**. The pipeline analyzes FDA databases and medical literature to detect drug-drug and drug-condition interactions using ontology-based reasoning.

### Key Features

- **Ontology Generation**: Creates safety ontologies for drug interactions
- **Multi-Source Correlation**: Correlates data from FDA databases and medical literature
- **Interaction Detection**: Detects drug-drug and drug-condition interactions
- **Safety Ontology**: Uses domain ontologies for safety analysis
- **Reasoning**: Emphasizes ontology and reasoning for interaction prediction

### Pipeline Architecture

1. **Phase 0**: Setup & Configuration
2. **Phase 1**: FDA Database & Medical Literature Ingestion
3. **Phase 2**: Drug Entity Extraction
4. **Phase 3**: Safety Ontology Generation
5. **Phase 4**: Multi-Source Correlation
6. **Phase 5**: Interaction Detection & Reasoning
7. **Phase 6**: Visualization & Export

---

## Installation


In [ ]:
%pip install -qU semantica networkx matplotlib plotly pandas groq


---

## Phase 0: Setup & Configuration


In [ ]:
import os
from semantica.core import Semantica, ConfigManager
from semantica.ontology import OntologyGenerator
from semantica.reasoning import GraphReasoner

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "your-key")

config_dict = {
    "project_name": "Drug_Interactions_Analysis",
    "extraction": {"provider": "groq", "model": "llama-3.1-8b-instant"},
    "knowledge_graph": {"backend": "networkx"},
    "ontology": {"base_uri": "https://drug-safety.example.org/ontology/"}
}

config = ConfigManager().load_from_dict(config_dict)
core = Semantica(config=config)
print("Configured for drug interactions analysis with ontology reasoning focus")


---

## Phase 1: Real Data Ingestion (Medical RSS Feeds)

Ingest drug interaction data from medical literature RSS feeds.


In [ ]:
from semantica.ingest import FeedIngestor, FileIngestor
import os

os.makedirs("data", exist_ok=True)

# Ingest from medical literature RSS feeds (real data sources)
medical_feeds = [
    "https://pubmed.ncbi.nlm.nih.gov/rss/search/1?term=drug+interaction&limit=10",
    # Add more medical feeds as needed
]

documents = []
for feed_url in medical_feeds:
    try:
        feed_ingestor = FeedIngestor()
        feed_documents = feed_ingestor.ingest(feed_url, method="rss")
        print(f"Ingested {len(feed_documents)} documents from {feed_url}")
        documents.extend(feed_documents)
    except Exception as e:
        print(f"Feed ingestion failed for {feed_url}: {e}")

# Fallback: Sample drug interaction data
if not documents:
    drug_data = """
    Warfarin interacts with Aspirin, increasing bleeding risk.
    Metformin should not be used with patients having kidney disease.
    Ibuprofen can interact with ACE inhibitors, reducing effectiveness.
    Statins may interact with grapefruit juice, increasing side effects.
    Digoxin interacts with Amiodarone, increasing toxicity risk.
    """
    with open("data/drug_interactions.txt", "w") as f:
        f.write(drug_data)
    documents = FileIngestor().ingest("data/drug_interactions.txt")
    print(f"Ingested {len(documents)} documents from sample data")


---

## Phase 2: Text Normalization & Advanced Chunking

Normalize drug names and use relation-aware chunking to preserve interaction triplets.


In [ ]:
from semantica.normalize import TextNormalizer
from semantica.split import TextSplitter

# Normalize drug names
normalizer = TextNormalizer()
normalized_documents = []
for doc in documents:
    normalized_text = normalizer.normalize(
        doc.content if hasattr(doc, 'content') else str(doc),
        clean_html=True,
        normalize_entities=True,
        remove_extra_whitespace=True
    )
    normalized_documents.append(normalized_text)

print(f"Normalized {len(normalized_documents)} documents")

# Use relation-aware chunking to preserve drug interaction triplets
relation_splitter = TextSplitter(
    method="relation_aware",
    chunk_size=1000,
    chunk_overlap=200
)

chunked_docs = []
for doc_text in normalized_documents:
    chunks = relation_splitter.split(doc_text)
    chunked_docs.extend([chunk.content if hasattr(chunk, 'content') else str(chunk) for chunk in chunks])

print(f"Created {len(chunked_docs)} relation-aware chunks (preserves interaction triplets)")


---

## Phase 3: Knowledge Graph Construction & Conflict Detection

Build drug knowledge graph and detect conflicts from multiple sources.


In [ ]:
from semantica.conflicts import ConflictDetector

# Build drug knowledge graph
result = core.build_knowledge_base(
    sources=chunked_docs,
    custom_entity_types=["Drug", "Interaction", "Condition", "SideEffect", "Safety"],
    graph=True
)

kg = result["knowledge_graph"]
entities = result["entities"]

# Detect conflicts in drug interaction data (e.g., conflicting interaction reports)
detector = ConflictDetector()
conflicts = detector.detect_conflicts(entities, kg.get("relationships", []))

print(f"Built drug KG with {len(kg.get('entities', []))} entities")
print(f"Detected {len(conflicts)} conflicts in drug interaction data")
if conflicts:
    resolved = detector.resolve_conflicts(conflicts, strategy="highest_confidence")
    print(f"Resolved {len(resolved)} conflicts")


---

## Phase 4: Safety Ontology Generation

Generate enhanced safety ontology from knowledge graph.


In [ ]:
# Generate safety ontology
ontology_gen = OntologyGenerator(base_uri="https://drug-safety.example.org/ontology/")
ontology = ontology_gen.generate_from_graph(kg)

print(f"Generated safety ontology with {len(ontology.get('classes', []))} classes")
print("Focus: Multi-source correlation, safety ontology, interaction detection, reasoning")


In [ ]:
# Use reasoning to detect interactions
reasoner = GraphReasoner(kg)
interactions = reasoner.find_patterns(pattern_type="interaction")

# Identify drug-drug interactions
drug_interactions = [r for r in kg.get("relationships", []) 
                     if "interact" in str(r.get("predicate", "")).lower()]

print(f"Interaction detection: {len(interactions)} interaction patterns found")
print(f"Drug interactions: {len(drug_interactions)} drug-drug interactions identified")
print("\n=== Pipeline Summary ===")
print(f"✓ Ingested {len(documents)} documents from medical RSS feeds")
print(f"✓ Normalized {len(normalized_documents)} documents")
print(f"✓ Created {len(chunked_docs)} relation-aware chunks")
print(f"✓ Detected and resolved {len(conflicts)} conflicts")
print(f"✓ Generated ontology with {len(ontology.get('classes', []))} classes")
print(f"✓ This cookbook emphasizes ontology reasoning, relation-aware chunking, and multi-source correlation")


---

## Phase 6: Visualization


In [ ]:
from semantica.visualization import KGVisualizer
from semantica.export import GraphExporter

visualizer = KGVisualizer()
visualizer.visualize(kg, output_path="drug_interactions_kg.html")

exporter = GraphExporter()
exporter.export(kg, format="rdf", output_path="drug_safety_ontology.ttl")

print("Drug interactions analysis complete")
print("Emphasizes: Multi-source correlation, safety ontology, interaction detection, reasoning")
